In [1]:
import requests
import json
from types import SimpleNamespace


import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
class Tags(object):
    
    def __init__(self, *initial_data, **kwargs):
              
        for dictionary in initial_data:
            
            for key in dictionary:

                if isinstance(dictionary[key], dict):
                    if len(dictionary[key])>1:
                        setattr(self, 'value', key)
                        setattr(self, key, Tags(dictionary[key]))
                    else:
                        setattr(self, key, Tags(dictionary[key]))
                else:
                    setattr(self, key, dictionary[key])
            
    def __repr__(self):
        return self.value.replace("__",":")
    
    def __str__(self):
        return self.value.replace("__",":")

In [4]:
kvs = osm.toolkit.get_osm_kvs()

In [5]:
tags = Tags(kvs)

In [27]:
area_Guernsey = "(49.4096, -2.6875,49.5194, -2.4901)"
area_Ealing = "(51.487, -0.351, 51.5335, -0.2655)"
area_britishisles = "(48.327, -12.063, 61.428, 1.78)"
area_Glasgow = "(55.5683, -4.9507, 56.1318, -3.6639)"
area_Whitelee = "(55.5, -4.4364, 55.7591, -4.1)"
area_Wales = "(51.221, -5.559, 53.619, -2.845)"
area_Shetlands = "(59.8034, -2.4527, 60.9331, -0.4257)"
area_London = "(51.448, -0.2554, 51.6061, 0.0714)"

In [28]:
df_stations = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.station,area=area_London)


In [53]:
df_stations["grouping"] = df_stations["line"].fillna(df_stations['network'])

In [54]:
df_stations['grouping'] = df_stations['grouping'].str.split(";",expand=True)[0]

In [57]:
output_file("London_Stations.html")

tooltips = {"tooltips":[("id", "@id"),("key", "@key"),("tag", "@tag"),("name", "@name"),("network", "@network"),("line", "@line"),("(lat,lon)", "@coordinates")]}

p = osm.toolkit.plot_latlon(df_stations,kwargs_for_figure=tooltips,kwargs_for_marker={"legend_group":"grouping"})

p.legend.visible = False

show(p)